Start jupyter server such that the print of the current working dir below is the root directory of the repo
Setup the environment...

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import sys
os.environ['STATIC_VARIABLES_FILE_PATH'] = "globalserver/static_variables.json"
os.environ['PATH_TO_GLOBALSERVER'] = "globalserver/api/"
if os.getcwd().split('/')[-1]!="Federated-Learning":
    os.chdir('..')
print(os.getcwd())
sys.path.append(os.getcwd())


from testing.test_class import Testing
from globalserver.operator_.operator_class_db import Operator

clients = [f"c{i}" for i in range(2)]
TestSetup = Testing(clients, start_servers=True, clear_logs=True, clear_db=False, interface=False)


/home/schneebi/PycharmProjects/federated-learning/Federated-Learning


2020-04-01 13:01:18,078 [MainProcess ] [INFO ] [test] [test_class.py / kill_servers / 194] KILLING SERVERS
2020-04-01 13:01:18,080 [MainProcess ] [INFO ] [test] [test_class.py / kill_global_server / 156] KILLING GLOBAL SERVER
2020-04-01 13:01:18,082 [MainProcess ] [INFO ] [test] [test_class.py / kill_client_interface_node / 163] KILLING CLIENT INTERFACE NODE
2020-04-01 13:01:18,084 [MainProcess ] [INFO ] [test] [test_class.py / kill_client_interface / 173] KILLING CLIENT INTERFACE
2020-04-01 13:01:18,085 [MainProcess ] [INFO ] [test] [test_class.py / kill_node_servers / 183] KILLINGCLIENTS
2020-04-01 13:01:21,242 [MainProcess ] [INFO ] [test] [test_class.py / start_global_server / 337] STARTING UP GLOBAL SERVER
2020-04-01 13:01:21,259 [MainProcess ] [INFO ] [test] [test_class.py / start_node_servers / 225] STARTING UP NODES


the commandline is ['/home/schneebi/PycharmProjects/federated-learning/venv/bin/python', 'api/globalserver_task_controller.py', 'C88A33B946']
the commandline is ['/home/schneebi/PycharmProjects/federated-learning/venv/bin/python', 'node_task_controller.py', 'c0', 'C88A33B946']
the commandline is ['/home/schneebi/PycharmProjects/federated-learning/venv/bin/python', 'node_task_controller.py', 'c1', 'C88A33B946']


Download the data from polybox https://polybox.ethz.ch/index.php/s/W1oSh3H81HqYSQp?path=%2FDatasets%2FKK_Box_Normalized_jsonl and put it into the datasets/kkbox folder. Then split the data into two sets of size 70000.

In [2]:
import random
import json

def split_data():
    
    with open(f'datasets/kk_box/train_kkbox.jsonl', 'r') as f:
        train=f.readlines()
    with open(f'datasets/kk_box/test_kkbox.jsonl', 'r') as f:
        test=f.readlines()
    random.shuffle(train)
    train_c1 = train[:70000]
    train_c2 = train[70000:140000]
    validation_c1 = train[140000:160000]
    validation_c2 = train[160000:180000]

    with open(f'datasets/train_c0.jsonl', 'w+') as f:
        f.writelines(train_c1)
    with open(f'datasets/validation_c0.jsonl', 'w+') as f:
        f.writelines(validation_c1)
    with open(f'datasets/test_c0.jsonl', 'w+') as f:
        f.writelines(validation_c1)

    with open(f'datasets/train_c1.jsonl', 'w+') as f:
        f.writelines(train_c2)
    with open(f'datasets/validation_c1.jsonl', 'w+') as f:
        f.writelines(validation_c2)
    with open(f'datasets/test_c1.jsonl', 'w+') as f:
        f.writelines(validation_c2)
        
# split_data()
#experiment_run_1




We define the experimental setup in several steps. First we define the model.

In [3]:
def kknox_nn(param_dict):
    model = Sequential()
    model.add(Dense(50, activation=tf.nn.relu, input_dim=61))
    model.add(Dense(50, activation=tf.nn.relu))
    model.add(Dense(1, activation=tf.nn.sigmoid))

    model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=param_dict['lr']),  # momentum=mt),
                  loss='binary_crossentropy',
                  metrics=[])
    model.summary()
    return model

Next we define the upkeep function. This function is run after every aggregation step. We simply want to shuffle the data. This is optional. Just set it to None below if you dont need.

In [4]:
def upkeep_function(self, experiment_document):
    import random
    with open(f'datasets/train_c0.jsonl', 'r') as f:
        train_c1 = f.readlines()
    random.shuffle(train_c1)
    with open(f'datasets/train_c0.jsonl', 'w+') as f:
        f.writelines(train_c1)

    with open(f'datasets/train_c1.jsonl', 'r') as f:
        train_c2 = f.readlines()
    random.shuffle(train_c2)
    with open(f'datasets/train_c1.jsonl', 'w+') as f:
        f.writelines(train_c2)

In [5]:
def preprocessing(batch):


    return batch

In [6]:
def stop_function(self, experiment_document, aggregated_metric):
        if len(experiment_document['validation_results'])>2  :
            return True, [aggregated_metric['AUC']]
        else:
            return False, [aggregated_metric['AUC']]

We define all the metadata as well as the training configurations. Most important: rounds, epochs, batch_size, steps_per_epoch.

In [7]:
setup_dict = {"model_function": #here we set how to run our model function. In the operator it will run kknox_nn({"lr": 0.1})
              {
    "function": kknox_nn,
    "parameters": {
        "lr": 0.20014587438532283
    }
},
    "git_version": 'e9339081b76ad3a89b1862bd38d8af26f0541f1c', # the git commit version (for now set manually)
    "protocol": 'NN', # the protocol name. NN is for neural networks
    "model_name": "test_model",
    "model_description": "this model is just to test the db",
    "testing": True, # This flag is helpful if you do not want to keep the created models. When you run Testing with cleardb=True it will clear all models and experiments with testing flag.
    "rounds": 2, # number of rounds of node-server communication to run
    "training_config": { # standard configuration for the training setup
        'epochs': 5,
        'verbose': 0,
        'batch_size': 10,
        "validation_steps": 12,#should be at most len(validation data)/batch_size
        "test_steps": 70, #should be at most  len(test data)/batch_size
        "steps_per_epoch": 70, #should be at most  len(training data)/batch_size

        "skmetrics":["f1_score"],
        "tfmetrics":["AUC","Accuracy"]
    },
   "clients": clients,
    "experiment_name": "kkbox",
    "experiment_description": f"desc if nice experiment",
        "preprocessing": {
    },
    "stop_function":stop_function
}

We define the hyperparameters we want to tune. Follow optuna doc: https://optuna.readthedocs.io/en/stable/reference/trial.html

In [8]:
tunable_parameters = [
    {
            'parameter_key_list': ['model_function', 'parameters', 'lr'],
            'function_name': 'suggest_uniform',
            'function_arguments': {'name': 'lr', 'low': 0.001, 'high': 0.5},
        }
]

And finally we define the objective we want to minimize. In this example we simply take the aggregated cross entropy loss we got in the last run.

In [9]:
def trial_loss_function(metrics):
    return metrics['aggregated_metric']['AUC']

This completes the setup.
We can now run the experiment.

In [10]:
operator = Operator()
import optuna

study = optuna.create_study()
study.optimize(lambda trial: operator.objective(trial, setup_dict, tunable_parameters, trial_loss_function),
               n_trials=5)

2020-04-01 13:01:22,571 [MainProcess ] [INFO ] [test] [operator_class_db.py / objective / 370] TEST WITH [{"parameter_key_list": ["model_function", "parameters", "lr"], "function_name": "suggest_uniform", "function_arguments": {"name": "lr", "low": 0.001, "high": 0.5}}]
2020-04-01 13:01:22,695 [MainProcess ] [DEBUG] [test] [cmd.py / execute / 719] Popen(['git', 'version'], cwd=/home/schneebi/PycharmProjects/federated-learning/Federated-Learning, universal_newlines=False, shell=None, istream=None)
2020-04-01 13:01:22,708 [MainProcess ] [DEBUG] [test] [cmd.py / execute / 719] Popen(['git', 'version'], cwd=/home/schneebi/PycharmProjects/federated-learning/Federated-Learning, universal_newlines=False, shell=None, istream=None)
2020-04-01 13:01:22,727 [MainProcess ] [DEBUG] [test] [cmd.py / execute / 719] Popen(['git', 'cat-file', '--batch-check'], cwd=/home/schneebi/PycharmProjects/federated-learning/Federated-Learning, universal_newlines=False, shell=None, istream=<valid stream>)
2020-04-

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                3100      
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 5,701
Trainable params: 5,701
Non-trainable params: 0
_________________________________________________________________


2020-04-01 13:01:37,799 [MainProcess ] [DEBUG] [test] [operator_utils.py / get_current_task / 94] Working on {'task_id': ObjectId('5e84748255fbdadaa7326ac3'), 'task_order': 1, 'task_name': 'train_model', 'task_status': 'scheduled'}
2020-04-01 13:01:42,807 [MainProcess ] [DEBUG] [test] [operator_utils.py / get_current_task / 94] Working on {'task_id': ObjectId('5e84748255fbdadaa7326ac6'), 'task_order': 4, 'task_name': 'aggregate', 'task_status': 'not_scheduled'}
2020-04-01 13:01:42,808 [MainProcess ] [INFO ] [test] [operator_class_db.py / aggregate / 250] Aggregating...
2020-04-01 13:01:42,861 [MainProcess ] [INFO ] [test] [operator_utils.py / aggregate_loss / 197] Average loss of {'f1_score': 0.48571428571428577, 'AUC': 0.761476457118988, 'Accuracy': 0.0} reached in this round.
2020-04-01 13:01:42,862 [MainProcess ] [INFO ] [test] [operator_class_db.py / _early_experiment_stop / 459] Please make sure that the stop function is described in the experiment
2020-04-01 13:01:47,871 [MainPro

ewrgfewrgr
{'aggregated_metric': {'f1_score': 0.600942850053759, 'AUC': 0.7792638540267944, 'Accuracy': 0.0}, 'additional_metrics': [0.7792638540267944]}
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 50)                3100      
_________________________________________________________________
dense_4 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 51        
Total params: 5,701
Trainable params: 5,701
Non-trainable params: 0
_________________________________________________________________


2020-04-01 13:02:13,135 [MainProcess ] [DEBUG] [test] [operator_utils.py / get_current_task / 94] Working on {'task_id': ObjectId('5e8474a655fbdadaa7326ad3'), 'task_order': 1, 'task_name': 'train_model', 'task_status': 'scheduled'}
2020-04-01 13:02:18,143 [MainProcess ] [DEBUG] [test] [operator_utils.py / get_current_task / 94] Working on {'task_id': ObjectId('5e8474a655fbdadaa7326ad6'), 'task_order': 4, 'task_name': 'aggregate', 'task_status': 'not_scheduled'}
2020-04-01 13:02:18,145 [MainProcess ] [INFO ] [test] [operator_class_db.py / aggregate / 250] Aggregating...
2020-04-01 13:02:18,163 [MainProcess ] [INFO ] [test] [operator_utils.py / aggregate_loss / 197] Average loss of {'f1_score': 0.4019607843137255, 'AUC': 0.7232657968997955, 'Accuracy': 0.0} reached in this round.
2020-04-01 13:02:18,164 [MainProcess ] [INFO ] [test] [operator_class_db.py / _early_experiment_stop / 459] Please make sure that the stop function is described in the experiment
2020-04-01 13:02:23,171 [MainPro

...

ewrgfewrgr
{'aggregated_metric': {'f1_score': 0.600942850053759, 'AUC': 0.7943803369998932, 'Accuracy': 0.0}, 'additional_metrics': [0.7943803369998932]}
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 50)                3100      
_________________________________________________________________
dense_10 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 51        
Total params: 5,701
Trainable params: 5,701
Non-trainable params: 0
_________________________________________________________________


2020-04-01 13:03:23,767 [MainProcess ] [DEBUG] [test] [operator_utils.py / get_current_task / 94] Working on {'task_id': ObjectId('5e8474ec55fbdadaa7326af3'), 'task_order': 1, 'task_name': 'train_model', 'task_status': 'scheduled'}
2020-04-01 13:03:28,775 [MainProcess ] [DEBUG] [test] [operator_utils.py / get_current_task / 94] Working on {'task_id': ObjectId('5e8474ec55fbdadaa7326af6'), 'task_order': 4, 'task_name': 'aggregate', 'task_status': 'not_scheduled'}
2020-04-01 13:03:28,776 [MainProcess ] [INFO ] [test] [operator_class_db.py / aggregate / 250] Aggregating...
2020-04-01 13:03:28,820 [MainProcess ] [INFO ] [test] [operator_utils.py / aggregate_loss / 197] Average loss of {'f1_score': 0.2, 'AUC': 0.7830513715744019, 'Accuracy': 0.0} reached in this round.
2020-04-01 13:03:28,821 [MainProcess ] [INFO ] [test] [operator_class_db.py / _early_experiment_stop / 459] Please make sure that the stop function is described in the experiment
2020-04-01 13:03:33,831 [MainProcess ] [DEBUG] 

ewrgfewrgr
{'aggregated_metric': {'f1_score': 0.5815910249872513, 'AUC': 0.7972713708877563, 'Accuracy': 0.0}, 'additional_metrics': [0.7972713708877563]}
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 50)                3100      
_________________________________________________________________
dense_13 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 51        
Total params: 5,701
Trainable params: 5,701
Non-trainable params: 0
_________________________________________________________________


2020-04-01 13:03:59,087 [MainProcess ] [DEBUG] [test] [operator_utils.py / get_current_task / 94] Working on {'task_id': ObjectId('5e84751055fbdadaa7326b03'), 'task_order': 1, 'task_name': 'train_model', 'task_status': 'scheduled'}
2020-04-01 13:04:04,095 [MainProcess ] [DEBUG] [test] [operator_utils.py / get_current_task / 94] Working on {'task_id': ObjectId('5e84751055fbdadaa7326b06'), 'task_order': 4, 'task_name': 'aggregate', 'task_status': 'not_scheduled'}
2020-04-01 13:04:04,097 [MainProcess ] [INFO ] [test] [operator_class_db.py / aggregate / 250] Aggregating...
2020-04-01 13:04:04,114 [MainProcess ] [INFO ] [test] [operator_utils.py / aggregate_loss / 197] Average loss of {'f1_score': 0.15384615384615383, 'AUC': 0.7481829822063446, 'Accuracy': 0.0} reached in this round.
2020-04-01 13:04:04,115 [MainProcess ] [INFO ] [test] [operator_class_db.py / _early_experiment_stop / 459] Please make sure that the stop function is described in the experiment
2020-04-01 13:04:09,123 [MainPr

ewrgfewrgr
{'aggregated_metric': {'f1_score': 0.5547619047619048, 'AUC': 0.800878643989563, 'Accuracy': 0.0}, 'additional_metrics': [0.800878643989563]}
